<a href="https://colab.research.google.com/github/aleschiavo94/Sketches-Web-Search-Engine/blob/main/code/siamese_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import numpy as np
import time
import itertools
import os, shutil
import tensorflow as tf
import matplotlib.pyplot as plt

from random import random
from ipywidgets import Image
from numpy.linalg import norm
from IPython.display import display
from keras.models import Model
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint

# tensorflow version 2.4.0
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, InceptionV3
from tensorflow.compat.v1.keras import backend as K

In [ ]:
zip_path = '/content/gdrive/Shareddrives/COMPUTER_VISION/MIM_zipped.zip'
!cp "{zip_path}" .
!unzip -q MIM_zipped.zip
!rm MIM_zipped.zip

### Setting up the project


### create Test (utile per quando faremo il training)

In [ ]:
#data loading
# reading from unzipped
BASE_DIR = "/content/content/gdrive/Shareddrives/COMPUTER_VISION/MIRCV"
# FILELIST_PATH = BASE_DIR + "/filelist.txt"
SKETCHES_DIR = os.path.join(BASE_DIR, "sketches")
MIRFLICKR_DIR = os.path.join(BASE_DIR, "mirflickr/mirflickr25k")

#print(FILELIST_PATH)

#num_folders = os.listdir(SKETCHES_DIR)
#print(num_folders)

In [ ]:
print(os.listdir(os.path.join(SKETCHES_DIR, "png")))
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

copytree(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/all')

['helmet', 'dolphin', 'floor lamp', 'mermaid', 'speed-boat', 'nose', 'trombone', 'bookshelf', 'present', 'comb', 'mug', 'bell', 'house', 'radio', 'satellite', 'alarm clock', 'streetlight', 'candle', 'boomerang', 'bicycle', 'rollerblades', 'pineapple', 'tooth', 'snail', 'butterfly', 'snake', 'skull', 'cell phone', 'parrot', 'umbrella', 'kangaroo', 'flower with stem', 'tractor', 'sword', 'door', 'fire hydrant', 'octopus', 'head', 'strawberry', 'bear (animal)', 't-shirt', 'bridge', 'face', 'cat', 'airplane', 'wine-bottle', 'flying saucer', 'cloud', 'van', 'windmill', 'shark', 'elephant', 'saxophone', 'space shuttle', 'sponge bob', 'mouth', 'pen', 'dog', 'shoe', 'syringe', 'tennis-racket', 'vase', 'rainbow', 'fish', 'tv', 'chandelier', 'sailboat', 'rifle', 'duck', 'angel', 'bush', 'truck', 'toothbrush', 'bee', 'feather', 'tree', 'door handle', 'church', 'tomato', 'trumpet', 'sun', 'suv', 'tire', 'piano', 'couch', 'cake', 'submarine', 'book', 'helicopter', 'screwdriver', 'mushroom', 'ship',

In [ ]:
test_images_per_class = 20 # le immagini sono al massimo 80
try:
  os.rename(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/train')
except:
  print('already renamed')

if not os.path.exists(f'{SKETCHES_DIR}/test'):
  os.mkdir(f'{SKETCHES_DIR}/test')
  for class_folder in os.listdir(f'{SKETCHES_DIR}/train'):
    if os.path.isdir(f'{SKETCHES_DIR}/train/{class_folder}'):
      images = os.listdir(f'{SKETCHES_DIR}/train/{class_folder}')
      os.mkdir(f'{SKETCHES_DIR}/test/{class_folder}')
      for i in range(test_images_per_class):
        os.rename(f'{SKETCHES_DIR}/train/{class_folder}/{images[i]}', f'{SKETCHES_DIR}/test/{class_folder}/{images[i]}')


### No_INDEX Class


In [ ]:
class NO_INDEX:
  def __init__(self):
    self.features = None
    self.ids = None
    self.labels = None

  def insert(self, features, ids, labels):
    assert features.shape[0] == len(ids), "mismatch between ids length and features"
    assert len(labels) == len(ids), "mismatch between ids length and labels"
    self.features = features
    self.ids = ids
    self.labels = labels
  
  def query(self, features, top_k, mode = 'euclidean', return_cost = False):
    assert mode in ['similarity', 'euclidean'], "mode must be similarity or euclidean"
    top_k += 1
    if mode == 'euclidean':
      dist = norm(self.features - np.array(features), axis=1)
      idx_partitioned = np.argpartition(dist, top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0] - 1)
      k = {}
      # qua è più comodo avere array 1- dimensionali
      k['ids'] = self.ids[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['labels'] = self.labels[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['distances'] = dist[idx_partitioned][:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      
      idx_sorted = np.argsort(k['distances'])
      idx_sorted = idx_sorted
      k['distances'] = k['distances'][idx_sorted]
      k['ids'] = k['ids'][idx_sorted]
      k['labels'] = k['labels'][idx_sorted]
      if return_cost:
        return (k, dist.size)
      return k
    sim = np.sum(self.features * np.array(features), axis=1) / (norm(self.features, axis=1) * norm(np.array([features]), axis=1))
    idx_partitioned = np.argpartition(sim, -(top_k - 1) if sim.shape[0] - 1 > top_k - 1 else sim.shape[0] - 1)
    k = {}
    # qua è più comodo avere array 1- dimensionali
    if sim.shape[0] - 1 > top_k - 1:  
      idx_partitioned = idx_partitioned[-(top_k - 1):]

    k['ids'] = self.ids[idx_partitioned].flatten()
    k['labels'] = self.labels[idx_partitioned].flatten()
    k['similarities'] = sim[idx_partitioned]
    idx_sorted = np.argsort(k['similarities'])[::-1]
    idx_sorted = idx_sorted
    k['similarities'] = k['similarities'][idx_sorted]
    k['ids'] = k['ids'][idx_sorted]
    k['labels'] = k['labels'][idx_sorted]
    if return_cost:
      return (k, sim.size)
    return k
    

### Loading images DATA


In [ ]:
# sketch image displaying 
filename = "airplane/1.png"
image1 = os.path.join(SKETCHES_DIR + '/all', filename)
display(Image.from_file(image1, width=229, height=229))

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04W\x00\x00\x04W\x08\x00\x00\x00\x00\x105\xd1!\x00\…

In [ ]:
# mirflickr image displaying 
filename = "png/im20.jpg"
image1 = os.path.join(MIRFLICKR_DIR, filename)
display(Image.from_file(image1, width=229, height=229))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x…

In [ ]:
IMG_HEIGHT = 299
IMG_WIDTH = 299
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
BATCH_SIZE = 64

In [ ]:

#yields batches of images from the subdirectories 
#found in the sketches directory [class_0 ... class_250]
#together with class labels.

#images normalization 
sketches_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
mirflickr_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

sketches_generator = sketches_datagen.flow_from_directory(
        # This is the target directory
        SKETCHES_DIR + '/all',
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)

mirflickr_generator = mirflickr_datagen.flow_from_directory(
        # This is the target directory
        MIRFLICKR_DIR,
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)


# unique, counts = np.unique(sketches_generator.classes, return_counts=True)
# labels_dict = dict(zip(unique, counts))
# print(labels_dict)

Found 20000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.


### Loading DNN for features extraction

In [ ]:
#Extracting features using the pretrained convolutional base 

def extract_features(extractor, generator, sample_count, dim=2048):
  features = np.zeros((sample_count, dim)) #extractor output shape 
  i = 0
  for inputs_batch, labels_batch in generator:
        
    start = time.time()
    features_batch = extractor.predict(inputs_batch)
    start = time.time()
    if (i + 1) * BATCH_SIZE > sample_count:
      features[i * BATCH_SIZE : sample_count , :] = features_batch
    else:
      features[i * BATCH_SIZE : (i + 1) * BATCH_SIZE, : ] = features_batch
    i += 1
    if i * BATCH_SIZE >= sample_count:
      break
  
  return features


### Fine tuning with classification

In [ ]:
MODEL_PATH = "/content/gdrive/Shareddrives/COMPUTER_VISION/models"

In [ ]:
a = np.array([1,2,3])
np.where(a > 0)

(array([0, 1, 2]),)

In [ ]:
def createDataGen(generator):
  # deve esserci lo shuffle
  while True:
    data = generator.next()
    data1 = generator.next()
    
    while data[0].shape[0] != data1[0].shape[0]:
      data = generator.next()
      data1 = generator.next()

    label = np.argmax(data[1], axis=1)
    label1 = np.argmax(data1[1], axis=1)
    permutation = np.zeros((label1.shape[0],)).astype(int)

    for i in range(label1.shape[0]):
      indices = np.where(label1 == label[i])[0]
      if indices.shape[0] > 0 and indices[0] not in permutation:
        permutation[i] = indices[0]
    
    for i in range(permutation.shape[0]):
      if i not in permutation:
        zeros = np.where(permutation == 0)[0]
        if zeros.shape[0] > 0 and permutation.shape[0] > zeros[0] and label1.shape[0] > i:
          permutation[zeros[0]] = i

    label_permuted = label1[permutation]
    data_permuted = data1[0][permutation]
    newLabel = np.equal(label, label_permuted).astype(int)

    # i want to create class balanced (almost)
    size = newLabel.shape[0] // 2

    ones = np.where(newLabel > 0)[0]
    if ones.shape[0] > size // 2:
      ones = ones[:size // 2] # prendo i primi n size
    
    final_index = np.concatenate((ones, np.ones((size - ones.shape[0])) * -1)).astype(int)
    while -1 not in final_index:
      to_change = np.where(final_index == -1)[0][0]
      z = np.where(newLabel == 0)[0]
      for i in range(z.shape[0]):
        if newLabel[z[i]] == 0 and z[i] not in final_index:
          final_index[to_change] = z[i]
    yield [data[0][final_index], data_permuted[final_index]], newLabel[final_index]

In [ ]:
IMG_HEIGHT = 299
IMG_WIDTH = 299
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
BATCH_SIZE = 64

In [ ]:
batch_training_size = 128 # così gli step sono di 64, quasi sempre 32 uno e 32 0
validation_split = 0.2

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    validation_split=validation_split,
    zoom_range=0.01,
    rotation_range=90,
    horizontal_flip = True
    ) # set validation split


train_generator = train_datagen.flow_from_directory(
    SKETCHES_DIR + '/train',
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_training_size,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    SKETCHES_DIR + '/train',
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_training_size,
    subset='validation') 

pretrained = models.load_model(os.path.join(MODEL_PATH, 'inception_finetuning_classification_2_moretrain_3.h5'))
for layer in pretrained.layers:
  layer.trainable = False

x = pretrained.layers[-2].output
output = layers.Dense(2048, activation='relu', name='extractor')(x)
cnn = Model(pretrained.input, output)

left_input = layers.Input(shape=INPUT_SHAPE)
right_input = layers.Input(shape=INPUT_SHAPE)

left_model = cnn(left_input)
right_model = cnn(right_input)

L1_layer = layers.Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
L1_distance = L1_layer([left_model, right_model])

# Add a dense layer with a sigmoid unit to generate the prediction
prediction = layers.Dense(1, activation='sigmoid')(L1_distance)

siamese = models.Model(inputs=[left_input, right_input], outputs=prediction)

model_checkpoint = ModelCheckpoint('checkpoint.h5', save_best_only=True, save_weights_only=True, verbose=1)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10)


siamese.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['acc'])
siamese.summary()
history = siamese.fit(
  createDataGen(train_generator),
  steps_per_epoch = (20000 - 250 * test_images_per_class) * (1 - validation_split) // batch_training_size * 2,
  epochs=10,
  validation_data = createDataGen(validation_generator),
  validation_steps =  (20000 - 250 * test_images_per_class) * validation_split // batch_training_size * 2,
  callbacks = [callback, model_checkpoint]
  )


siamese.load_weights('checkpoint.h5')
models.save_model(siamese, os.path.join(MODEL_PATH, 'inception_finetuning_siamese.h5'))


Found 12000 images belonging to 250 classes.
Found 3000 images belonging to 250 classes.
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
model_3 (Functional)            (None, 2048)         34391840    input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lam

In [ ]:
batch_training_size = 128 # così gli step sono di 64, quasi sempre 32 uno e 32 0
validation_split = 0.2

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    validation_split=validation_split,
    zoom_range=0.01,
    rotation_range=90,
    horizontal_flip = True
    ) # set validation split


train_generator = train_datagen.flow_from_directory(
    SKETCHES_DIR + '/train',
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_training_size,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    SKETCHES_DIR + '/train',
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_training_size,
    subset='validation') 

siamese = models.load_model(os.path.join(MODEL_PATH, 'inception_finetuning_siamese.h5'))

siamese.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['acc'])
siamese.summary()

model_checkpoint = ModelCheckpoint('checkpoint.h5', save_best_only=True, save_weights_only=True, verbose=1)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = siamese.fit(
  createDataGen(train_generator),
  steps_per_epoch = (20000 - 250 * test_images_per_class) * (1 - validation_split) // batch_training_size * 2,
  epochs=10,
  validation_data = createDataGen(validation_generator),
  validation_steps =  (20000 - 250 * test_images_per_class) * validation_split // batch_training_size * 2,
  callbacks = [callback, model_checkpoint]
  )

siamese.load_weights('checkpoint.h5')
models.save_model(siamese, os.path.join(MODEL_PATH, 'inception_finetuning_siamese_2.h5'))



Found 12000 images belonging to 250 classes.
Found 3000 images belonging to 250 classes.
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
model_3 (Functional)            (None, 2048)         34391840    input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lam

In [ ]:
loaded = models.load_model(os.path.join(MODEL_PATH, 'inception_finetuning_siamese_2.h5'))
finetuned_extactor = Model(loaded.layers[2].input, loaded.layers[-3].output)
finetuned_extactor.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, None, None, 3 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, None, None, 3 96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, None, None, 3 0           batch_normalization_470[0][0]    
______________________________________________________________________________________________

In [ ]:
sketches_features_finetuned = extract_features(finetuned_extactor, sketches_generator, 20000)
mirflickr_features_finetuned = extract_features(finetuned_extactor, mirflickr_generator, 25000)

In [ ]:
no_index_finetuned = NO_INDEX()
no_index_finetuned.insert(np.vstack((sketches_features_finetuned, mirflickr_features_finetuned)), 
                np.concatenate((sketches_generator.filenames, mirflickr_generator.filenames)), 
                np.concatenate((sketches_generator.labels, np.array([250] * mirflickr_features_finetuned.shape[0]))))

In [ ]:
query_result_ed = no_index_finetuned.query(sketches_features_finetuned[0], 80)
query_result_sim = no_index_finetuned.query(sketches_features_finetuned[0], 80, mode='similarity')

In [ ]:
print(sketches_features_finetuned.shape)
print(mirflickr_features_finetuned.shape)

(20000, 2048)
(25000, 2048)


In [ ]:
print(query_result_ed)
print(query_result_sim)

{'ids': array(['airplane/1.png', 'airplane/46.png', 'airplane/48.png',
       'airplane/20.png', 'airplane/65.png', 'airplane/75.png',
       'airplane/37.png', 'airplane/68.png', 'airplane/54.png',
       'airplane/69.png', 'airplane/44.png', 'airplane/8.png',
       'airplane/12.png', 'airplane/53.png', 'airplane/42.png',
       'airplane/38.png', 'airplane/62.png', 'airplane/50.png',
       'airplane/63.png', 'airplane/74.png', 'airplane/49.png',
       'airplane/21.png', 'airplane/18.png', 'airplane/40.png',
       'airplane/14.png', 'airplane/32.png', 'airplane/17.png',
       'airplane/28.png', 'airplane/58.png', 'airplane/29.png',
       'airplane/31.png', 'airplane/80.png', 'airplane/72.png',
       'airplane/52.png', 'shark/14777.png', 'space shuttle/15826.png',
       'airplane/30.png', 'airplane/56.png', 'airplane/70.png',
       'airplane/36.png', 'shark/14753.png', 'shark/14729.png',
       'dolphin/5383.png', 'shark/14764.png', 'airplane/33.png',
       'fish/6567.png', '

### Evaluation

In [ ]:
def average_precision(requested_label, result_labels, n_ground_truth = 80):
  """
  label ricercata, label ottenute, il numero di oggetti che ci sono quella label
  """
  correct_array = (requested_label == result_labels).astype(int)
  precision_array = [np.mean(correct_array[:k]) for k in range(1, correct_array.shape[0] + 1)]
  # print(precision_array * correct_array) # mi rimangono solo quelli a 1
  return np.sum(precision_array * correct_array) / n_ground_truth

print(average_precision(1, np.array([1,0,1,1,1,1,0,0,0,1]), 6)) # importante cambiare l'ultimo parametro con il numero massimo di true label
print(average_precision(1, np.array([0,1,0,0,1,1,1,0,1,0]), 5)) # importante cambiare l'ultimo parametro con il numero massimo di true label
print(average_precision(1, np.array([1,0,1,1,1,1,0,0]), 6)) # importante cambiare l'ultimo parametro con il numero massimo di true label
print(average_precision(1, np.array([0,1,0,0,1,1,1,0]), 5)) # importante cambiare l'ultimo parametro con il numero massimo di true label
print(average_precision(1, np.array([1,1,1,1, 0 ,0,0]), 4))

0.7749999999999999
0.5053968253968254
0.6749999999999999
0.3942857142857143
1.0


In [ ]:
def mAP(index, features, n_queries = 250, n_labels = 250, img_per_labels = 80, mode = 'euclidean'):
  sum = 0
  for i in range(n_queries):
    label = i % n_labels
    image_idx = ((i * img_per_labels) + int(random() * img_per_labels)) % (n_labels * img_per_labels)
    # print('QUERY')
    # print('index = ' + str(image_idx))
    # print('label =' + str(label))
    res = index.query(features[image_idx], img_per_labels, mode = mode)
    # print('first label of resultset (must be equal to label) = ' + str(res['labels'][0]))
    assert res['labels'][0] == label, 'deve essere della stessa label'
    sum += average_precision(label, res['labels'], img_per_labels)
  return sum / n_queries


In [ ]:
map_value = mAP(no_index_finetuned, sketches_features_finetuned)
print(map_value)

0.28116778789511576


In [ ]:
map_value = mAP(no_index_finetuned, sketches_features_finetuned, mode='similarity')
print(map_value)

0.33816336124771507
